In [1]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF4'

Mounted at /content/drive


In [ ]:
import sklearn
#import scikeras
import tensorflow

print("scikit-learn version:", sklearn.__version__)
#print("scikeras version:", scikeras.__version__)
print("tensorflow version:", tensorflow.__version__)


scikit-learn version: 1.6.1
tensorflow version: 2.18.0


In [1]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1


In [2]:
pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.9 MB/s eta 0:00:00


In [ ]:
import os
os._exit(00)

In [2]:
!pip install scikeras

In [3]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import Dense, GRU
from tensorflow.keras.layers import Dense, Flatten, LSTM, Bidirectional, Input, Dropout, InputLayer, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [4]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [5]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])
df_META = df_META.sort_values('date').reset_index(drop=True)

In [6]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])
df_AAPL = df_AAPL.sort_values('date').reset_index(drop=True)

In [7]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])
df_MSFT = df_MSFT.sort_values('date').reset_index(drop=True)

In [8]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])
df_NVDA = df_NVDA.sort_values('date').reset_index(drop=True)

In [6]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])
df_GOOG = df_GOOG.sort_values('date').reset_index(drop=True)

In [7]:
train_size = int(len(df_META) * 0.75)
test_size = len(df_META) - train_size

# META

## Train Test

In [8]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [9]:
X_META = train_META.drop(columns=["Stockprice_META","date","sentiment_META","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

#Normalized

In [13]:
y_META

,Stockprice_META
0,203.49
1,208.11
2,202.97
3,200.06
4,187.47
...,...
559,464.63
560,467.78
561,465.78
562,478.22


In [14]:
train_META

,date,Stockprice_META,sentiment_META,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,RETAIL_SALES,US_inflation_rate,Real_GDP,US_Dollar_Index,S&P500,NASDAQ,WTI,BTC,Gold
0,2022-03-01,203.49,0.000000,0.08,3.7,287.504,1.72,595064.0,0.075,5337.277,97.41,4306.26,13532.46,107.77,44420.3,1943.8
1,2022-03-02,208.11,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.39,4386.54,13752.02,112.22,43912.8,1922.3
2,2022-03-03,202.97,0.000000,0.08,3.7,287.504,1.86,595064.0,0.075,5337.277,97.79,4363.49,13537.94,110.52,42463.0,1935.9
3,2022-03-04,200.06,0.000000,0.08,3.7,287.504,1.74,595064.0,0.075,5337.277,98.65,4328.87,13313.44,116.37,39142.7,1966.6
4,2022-03-07,187.47,0.000000,0.08,3.7,287.504,1.78,595064.0,0.075,5337.277,99.29,4201.09,12830.96,122.09,38024.8,1995.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2024-05-21,464.63,0.129250,5.33,4.0,314.069,4.41,643843.0,0.034,5817.169,104.66,5321.41,16832.62,78.69,70139.9,2425.9
560,2024-05-22,467.78,0.622400,5.33,4.0,314.069,4.43,643843.0,0.034,5817.169,104.93,5307.01,16801.54,77.53,69155.4,2392.9
561,2024-05-23,465.78,0.399033,5.33,4.0,314.069,4.47,643843.0,0.034,5817.169,105.11,5267.84,16736.03,77.22,67975.7,2337.2
562,2024-05-24,478.22,0.437117,5.33,4.0,314.069,4.46,643843.0,0.034,5817.169,104.72,5304.72,16920.79,78.18,68547.6,2334.5


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_META = scaler.fit_transform(X_META)

In [16]:
X_META

array([[0.        , 0.5       , 0.        , ..., 0.48764065, 0.49998342,
        0.39293165],
       [0.        , 0.5       , 0.        , ..., 0.51989763, 0.49112499,
        0.36654804],
       [0.        , 0.5       , 0.        , ..., 0.48844576, 0.4658187 ,
        0.38323721],
       ...,
       [1.        , 1.        , 1.        , ..., 0.95829777, 0.91114346,
        0.87569027],
       [1.        , 1.        , 1.        , ..., 0.98544205, 0.92112599,
        0.87237698],
       [1.        , 1.        , 1.        , ..., 1.        , 0.91795616,
        0.89937416]])

In [17]:
X_META.shape[0]

564

In [18]:
look_back = 10

X_META1, y_META1 = [], []
for i in range(len(X_META) - look_back):
    X_META1.append(X_META[i:i + look_back])
    y_META1.append(y_META[i + look_back])

X_META1, y_META1 = np.array(X_META1), np.array(y_META1)

In [19]:
y_META1.shape

(554,)

In [20]:
X_META1.shape

(554, 10, 10)

In [11]:
tscv = TimeSeriesSplit(n_splits=5)

#MODEL

In [22]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(GRU(256, input_shape=(look_back, X_META1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer=optimizer)
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_META1, y_META1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_


y_pred = best_model.predict(X_META1)
mse = mean_squared_error(y_META1, y_pred)
rmse = root_mean_squared_error(y_META1, y_pred)
mae = mean_absolute_error(y_META1, y_pred)
mape = mean_absolute_percentage_error(y_META1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'optimizer': 'AdamW'}
Test Set Mean Squared Error: 102.9789
Test Set Root Mean Squared Error: 10.1479
Test Set Mean Absolute Error: 7.8139
Test Set Mean Absolute Percentage Error: 0.0361


#APPL

In [23]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [24]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","sentiment_AAPL", "date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [25]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [26]:
look_back = 10

X_AAPL1, y_AAPL1 = [], []
for i in range(len(X_AAPL) - look_back):
    X_AAPL1.append(X_AAPL[i:i + look_back])
    y_AAPL1.append(y_AAPL[i + look_back])

X_AAPL1, y_AAPL1 = np.array(X_AAPL1), np.array(y_AAPL1)

In [27]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(GRU(256, input_shape=(look_back, X_AAPL1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_AAPL1, y_AAPL1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_AAPL1)
mse = mean_squared_error(y_AAPL1, y_pred)
rmse = root_mean_squared_error(y_AAPL1, y_pred)
mae = mean_absolute_error(y_AAPL1, y_pred)
mape = mean_absolute_percentage_error(y_AAPL1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 200, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 10.1991
Test Set Root Mean Squared Error: 3.1936
Test Set Mean Absolute Error: 2.5423
Test Set Mean Absolute Percentage Error: 0.0154


#MSFT

In [28]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [29]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","sentiment_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [30]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [31]:
look_back = 10

X_MSFT1, y_MSFT1 = [], []
for i in range(len(X_MSFT) - look_back):
    X_MSFT1.append(X_MSFT[i:i + look_back])
    y_MSFT1.append(y_MSFT[i + look_back])

X_MSFT1, y_MSFT1 = np.array(X_MSFT1), np.array(y_MSFT1)

In [32]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(GRU(256, input_shape=(look_back, X_MSFT1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)

param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}


grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_MSFT1, y_MSFT1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_MSFT1)
mse = mean_squared_error(y_MSFT1, y_pred)
rmse = root_mean_squared_error(y_MSFT1, y_pred)
mae = mean_absolute_error(y_MSFT1, y_pred)
mape = mean_absolute_percentage_error(y_MSFT1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 32, 'epochs': 200, 'optimizer': 'RMSprop'}
Test Set Mean Squared Error: 33.6580
Test Set Root Mean Squared Error: 5.8016
Test Set Mean Absolute Error: 4.5952
Test Set Mean Absolute Percentage Error: 0.0153


#NVDA

In [15]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [16]:
X_NVDA = train_NVDA.drop(columns=["Stockprice_NVDA","sentiment_NVDA","date", "Interest","RETAIL_SALES", "US_Dollar_Index"])
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [17]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [18]:
look_back = 10

X_NVDA1, y_NVDA1 = [], []
for i in range(len(X_NVDA) - look_back):
    X_NVDA1.append(X_NVDA[i:i + look_back])
    y_NVDA1.append(y_NVDA[i + look_back])

X_NVDA1, y_NVDA1 = np.array(X_NVDA1), np.array(y_NVDA1)

In [19]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(GRU(256, input_shape=(look_back, X_NVDA1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)

param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_NVDA1, y_NVDA1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_NVDA1)
mse = mean_squared_error(y_NVDA1, y_pred)
rmse = root_mean_squared_error(y_NVDA1, y_pred)
mae = mean_absolute_error(y_NVDA1, y_pred)
mape = mean_absolute_percentage_error(y_NVDA1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 16, 'epochs': 150, 'optimizer': 'Nadam'}
Test Set Mean Squared Error: 3.5115
Test Set Root Mean Squared Error: 1.8739
Test Set Mean Absolute Error: 1.3178
Test Set Mean Absolute Percentage Error: 0.0419


#GOOG

In [12]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [13]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL", "sentiment_GOOGL", "date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [14]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [15]:
look_back = 10

X_GOOG1, y_GOOG1 = [], []
for i in range(len(X_GOOG) - look_back):
    X_GOOG1.append(X_GOOG[i:i + look_back])
    y_GOOG1.append(y_GOOG[i + look_back])

X_GOOG1, y_GOOG1 = np.array(X_GOOG1), np.array(y_GOOG1)

In [16]:
def create_model(optimizer='Adam'):
    model = Sequential()
    model.add(GRU(256, input_shape=(look_back, X_GOOG1.shape[2])))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mse', optimizer='Adam')
    return model

model = KerasRegressor(model=create_model, verbose=0)


param_grid = {'optimizer': ['RMSprop', 'Adam', 'AdamW', 'Nadam'],
              'batch_size' :[16,32,64,128,256],
              'epochs' :[60,80,100,150,200]}



grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs = -1)


grid_result = grid.fit(X_GOOG1, y_GOOG1)

best_model = grid_result.best_estimator_
best_params = grid_result.best_params_

y_pred = best_model.predict(X_GOOG1)
mse = mean_squared_error(y_GOOG1, y_pred)
rmse = root_mean_squared_error(y_GOOG1, y_pred)
mae = mean_absolute_error(y_GOOG1, y_pred)
mape = mean_absolute_percentage_error(y_GOOG1, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Best Hyperparameters: {'batch_size': 32, 'epochs': 150, 'optimizer': 'Adam'}
Test Set Mean Squared Error: 26.1744
Test Set Root Mean Squared Error: 5.1161
Test Set Mean Absolute Error: 4.1688
Test Set Mean Absolute Percentage Error: 0.0338
